In [2]:
!pip install python-dotenv
!pip install openai

In [4]:
import requests
import json
import os
from dotenv import load_dotenv

load_dotenv()  # Load variables from .env file
mergeflow_api_key = os.getenv('MERGEFLOW_API_KEY')

# the API call
query = 'factory automation' # topic is "factory automation"
dataset = '&sp=3814' # dataset: industry news
rows = str(3) # first 3 docs only

# in the url below, replace "fw" by your user name. You can see your user name at mergeflow.net
# -> Account -> the string in angle brackets
url = 'https://mergeflow.net/api/v1/leonard/search/getContent?q=' + query + dataset + '&rows=' + rows

# Define the headers to be sent with the request
headers = {
    'MergeflowNet-Api-Auth-Key': mergeflow_api_key
}

# Get and print the response
response = requests.post(url, headers=headers)

mergeflow_docs = []

# Check the response status code and content
if response.status_code == 200:
    mergeflow_response_json = response.json()
    
    #pretty-print the JSON response -- useful to understand what the JSON looks like
    #pretty_json = json.dumps(mergeflow_response_json, indent=4)
    #print(pretty_json)
    
    # get title, URL, date, and content
    for document in mergeflow_response_json['Documents']:
        current_doc = {
            'title': document['Title'],
            'date': document['Date'],
            'url': document['Url'],
            'content': document['Content'][:8000]
        }
        
        mergeflow_docs.append(current_doc)
        #print(content)        

    print('Found ' + str(len(mergeflow_docs)) + ' documents.')

else:
    print('Request failed with status code:', response.status_code)
    print(response.text)  # Print the response text




Found 3 documents.


In [5]:
import openai

openai.api_key = os.getenv('OPENAI_API_KEY')

system_prompt = f"""You are Senior Vice President Technology and Innovation & CTO of Factory Automation. When you evaluate a new business idea, you strictly adhere to the following guiding principles:
        - Prioritize quick, low-investment idea testing.
        - Aim for large profits with smaller budgets, avoiding investments over 10-15 million per portfolio element.
        - Target a minimum 10 million EUR annual revenue.
        - Specialize in LLMs in Engineering and Operations.
        - Avoid new ecosystem creation.
        - Prioritize product over solution business for scalability.
        - Consider AI as a potential product.
        - Ensure all initiatives align with business profitability.
        - Aim for non-cannibalizing, fringe markets.
        
        You are now facing the following problem: Lack of real-time visibility and control over production processes
        Pain Point: Manufacturing companies face inefficiency and delays in production due to a lack of real-time visibility and control over their processes, leading to increased costs and missed deadlines.
        Most Affected Customer: Medium to large-sized manufacturing companies.
        Reasoning: This problem affects a significant number of customers and can be addressed by providing real-time visibility and control, improving overall efficiency and responsiveness in production processes.

        """

budgets = [100, 10000, 1000000]  # Budget constraints for each solution proposal

for doc in mergeflow_docs:
    user_prompt = f"""
        Based on each budget of {budgets} and the problem of inefficiency and delays in production processes described earlier, come up with three different solutions that are each tailored to this budget. 
        The solutions should be innovative and be achievable within the budget.
        At each iteration, you should leverage insights or approaches mentioned in the following document excerpt:

        Document excerpt:
        ```
        {doc['content']}
        ```

        Please make sure you use the budgets that were set correctly, and that the idea is really achievable within that budget.
        Critically evaluate the feasibility of implementing this solution within the given budget and its potential impact on addressing the production inefficiencies. 
        Please also give a detailed explanation of how the technology is supposed to work and be tailored to the encountered problem.
        Rate the idea from 1 (bad) to 3 (good), and provide a brief explanation.
        """
    
    
    completion = openai.chat.completions.create(
        model="gpt-3.5-turbo-0613",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        temperature=0.7,
        max_tokens=2000,
        top_p=0.95,
        frequency_penalty=0,
        presence_penalty=0,
        stop=None
    )

    print(completion.choices[0].message.content) # This prints the content of the output in a more readable way
    print("\n---\n")  # This adds a separator between the messages for readability



Solution 1: Real-time Monitoring and Control System (Budget: 100 EUR)
Feasibility: It is not feasible to implement a real-time monitoring and control system with a budget of only 100 EUR. The cost of hardware, software, and infrastructure required to develop such a system would far exceed this budget.

Explanation: A real-time monitoring and control system would involve the use of sensors, data collection devices, and software to gather real-time data from various stages of the production process. This data would then be analyzed and used to provide insights and control over the production processes. The system would allow manufacturers to monitor key performance indicators, identify bottlenecks, and make real-time adjustments to optimize production efficiency.

Impact: Implementing a real-time monitoring and control system would greatly improve visibility and control over production processes, leading to reduced inefficiencies and delays. Manufacturers would be able to identify and ad

Solution 1: Real-time Monitoring and Analytics System (Budget: 100 EUR)
Feasibility: This solution can be implemented within the given budget as it primarily involves the development and deployment of a software system for real-time monitoring and analytics. The cost of software development and integration can be managed within the budget.

Description:
The real-time monitoring and analytics system will involve the installation of sensors and data collection devices in the production processes of manufacturing companies. These sensors will gather data on various parameters such as machine performance, production rates, and quality metrics. The data will be transmitted to a centralized software system that will analyze and visualize the information in real-time.

The software system will provide a dashboard that displays key performance indicators (KPIs) and alerts manufacturing managers and operators about any deviations or inefficiencies in the production processes. It will also gener

In [6]:
import openai

openai.api_key = os.getenv('OPENAI_API_KEY')

system_prompt = f"""You are Senior Vice President Technology and Innovation & CTO of Factory Automation. When you evaluate a new business idea, you strictly adhere to the following guiding principles:
        - Prioritize quick, low-investment idea testing.
        - Aim for large profits with smaller budgets, avoiding investments over 10-15 million per portfolio element.
        - Target a minimum 10 million EUR annual revenue.
        - Specialize in LLMs in Engineering and Operations.
        - Avoid new ecosystem creation.
        - Prioritize product over solution business for scalability.
        - Consider AI as a potential product.
        - Ensure all initiatives align with business profitability.
        - Aim for non-cannibalizing, fringe markets.
        
        You are now facing the following problem: Lack of real-time visibility and control over production processes
        Pain Point: Manufacturing companies face inefficiency and delays in production due to a lack of real-time visibility and control over their processes, leading to increased costs and missed deadlines.
        Most Affected Customer: Medium to large-sized manufacturing companies.
        Reasoning: This problem affects a significant number of customers and can be addressed by providing real-time visibility and control, improving overall efficiency and responsiveness in production processes.

        """

budgets = [100, 10000, 1000000]  # Budget constraints for each solution proposal

    
user_prompt = f"""
    Based on each budget of {budgets} and the problem of inefficiency and delays in production processes described earlier, come up with three different solutions that are each tailored to this budget. 
    The solutions should be innovative and be achievable within the budget.
    You should leverage insights or approaches mentioned in the following document excerpts:

    Document excerpt:
    ```
    {doc['content']}
    ```
    In coming up with ideas, use insights from the different excerpts (that are from industry news).
    
    Please make sure you use the budgets that were set correctly, and that the idea is really achievable within that budget.
    
    Critically evaluate the feasibility of implementing this solution within the given budget and its potential impact on addressing the production inefficiencies. 
    
    Please also give a very detailed explanation of how the technology is supposed to work and be tailored to the encountered problem. Please provide some sources of inspiration that you used from the document excerpts.
    
    Rate the idea from 1 (bad) to 3 (good), and provide a brief explanation.
    """
    
    
completion = openai.chat.completions.create(
        model="gpt-3.5-turbo-0613",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        temperature=0.7,
        max_tokens=2000,
        top_p=0.95,
        frequency_penalty=0,
        presence_penalty=0,
        stop=None
    )

print(completion.choices[0].message.content) # This prints the content of the output in a more readable way
print("\n---\n")  # This adds a separator between the messages for readability


Solution 1: Real-time Production Monitoring System (Budget: 100 EUR)
- Idea: Develop a basic real-time production monitoring system using low-cost sensors and cloud-based data storage.
- Feasibility: This solution can be implemented within a budget of 100 EUR by utilizing affordable sensors and leveraging existing cloud-based platforms for data storage and analysis.
- Technology: The system will consist of sensors placed at key points in the production process to collect data on parameters such as temperature, pressure, and machine status. This data will be transmitted wirelessly to a cloud-based platform where it will be stored and analyzed in real-time. The platform will provide a user-friendly interface for operators to monitor the production process and receive alerts in case of any deviations or issues.
- Tailored to the Problem: The real-time production monitoring system will provide manufacturing companies with visibility and control over their production processes by continuous